<a href="https://colab.research.google.com/github/zzzzzuuuuu/big-data-analytics/blob/main/Multi_Logistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 다중 로지스틱 회귀 모델
- 다중 클래스 분류 (multi-class calssfication)

### 다중 로지스틱 회귀 (multinomial or softmax 로지스틱 회귀)
- 3차원인 경우
- 학습데이터 셋은 각 행마다 (1,0,0), (0,1,0), (0,0,1)
- 가설함수 p.8, 비용함수, 손실함수 p.9 참고
- 이진 로지스틱 회귀에서처럼 교차 엔트로피 활용

### 이론적 배경
- 로짓의 확장 *p.10
  - 선형 모델을 로짓 개수(클래스 개수)만큼 가정하여 각 로짓값을 선형으로 모델링
  - 시그모이드로부터 소프트맥스(softmax)로의 확장
  - 시그모이드 함수가 로짓의 역함수여서 로짓이 입력이 됐을 때, p를 출력했던 것처럼
  - 소프트맥스 함수는 j번째 로짓을 p_{j}로 만듦
  - 다중 로지스틱 회귀에서는 소프트맥스 함수를 활용

### 다중 로지스틱 회귀 예제
- scikit-learn의 함수를 활용하여 다중 로지스틱 회귀를 적용
  - LogisticRegression 객체 옵션 - #1
    - penalty: 모델의 복잡도를 조절하는 방법
    - C: 패널티 정도
    - solver: 다양한 해결 방법을 고를 수 있음
    - max_iter: 반복 횟수
    - mulit_class: 'auto', 'ovr', 'multinomial'
      - 'ovr': 이진 로지스틱 회귀를 모든 라벨에 대해 적용
      - **'multinomial': softmax 로지스틱 회귀**
      - 'auto': 데이터에 따라 자동으로 적용 (default)

    - LogisticRegression 객체 속성
      - classes: 클래스 리스트
      - n_features_in_: 피쳐 개수
      - coef_: (클래스 개수, 피쳐 개수)만큼의 계수를 반환
      - intercept_: (클래스 개수)만큼의 절편을 반환

## 1. 다중 로지스틱 회귀 예제

### breast cancer dataset

In [1]:
import numpy as np
from sklearn.datasets import load_breast_cancer # 사실은 이진로지스터회귀임.. 예시 가져옴
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X, y = load_breast_cancer(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [2]:
clf = LogisticRegression(random_state=1234, solver='sag', penalty='l2', C=10,
                         max_iter=10000, multi_class='multinomial')
clf.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(C=10, max_iter=10000, multi_class='multinomial',
                   random_state=1234, solver='sag')

In [4]:
y_pred_test = clf.predict(X_test_scaled)
print(f'정확도: {(y_pred_test == y_test).mean() * 100: .2f}%')

정확도:  96.28%


### iris dataset

In [5]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [6]:
X, y = load_iris(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=1234)

In [7]:
scaler = MinMaxScaler() # 최소-최대 스케일러, 데이터를 0~1 사이의 값으로 정규화
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
# multi_class='multinomial'로 다중 로지스틱 회귀를 활성화
clf = LogisticRegression(random_state=1234, solver='sag', penalty='l2', C=10,
                         max_iter=10000, multi_class='multinomial')
clf.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(C=10, max_iter=10000, multi_class='multinomial',
                   random_state=1234, solver='sag')

In [12]:
y_pred_test = clf.predict(X_test_scaled)
print(f'정확도: {(y_pred_test == y_test).mean() * 100: .2f}%')

정확도:  96.00%


### tips를 4분위로 이산화하고 나머지 피쳐를 인풋으로 분류하는 모델을 학습

In [19]:
import pandas as pd
tips = pd.read_csv("./tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


## 분류의 성능 지표
### 회귀의 성능 지표 vs. 분류의 성능 지표
- 회귀 - #2
  - 평균절댓값오차, MAE(Mean absolute error)
  - 루트평균제곱오차, RMSE(Root mean squared error)
  - R2 score (분류기의 score method로 얻을 수 있음)

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes
from sklearn.linear_model import SGDRegressor

X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=1234)

# 경사하강법(SGD)
reg = SGDRegressor(penalty=None, max_iter=10000, learning_rate='constant', eta0=0.1)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_train)
y_pred_test = reg.predict(X_test)

print(f'테스트 데이터셋 MAE: {np.abs(y_pred_test - y_test).mean()}')
print(f'test dataset MSE: {(y_pred_test - y_test**2).mean()}')
print(f'test dataset RMSE: {np.sqrt((y_pred_test - y_test)**2).mean()}')
print(f'test dataset R2 score: {reg.score(X_test, y_test)}')

테스트 데이터셋 MAE: 43.12025292459359
test dataset MSE: -29460.23056721258
test dataset RMSE: 43.12025292459359
test dataset R2 score: 0.5017390123097338


In [24]:
tips = pd.read_csv("./tips.csv")
tips.head()
tips['smoker']=tips['smoker'].map({'No':0,"Yes":1})
tips['day']=tips['day'].map({'Thur':0,"Fri":1,"Sat":2,"Sun":3})
tips['time']=tips['time'].map({'Lunch':0,"Dinner":1})
X_train, X_test, y_train, y_test = train_test_split(tips.drop(columns="tip"),tips.tip,test_size=0.33,random_state=1234)
reg = SGDRegressor(penalty=None, max_iter=10000, learning_rate='constant', eta0=0.001)
reg = reg.fit(X_train, y_train)

y_pred = reg.predict(X_test)
print('테스트 데이터셋 MAE:',np.abs(y_pred - y_test).mean())
print('테스트 데이터셋 MSE:',((y_pred - y_test)**2).mean())
print('테스트 데이터셋 RMSE:',np.sqrt(((y_pred - y_test)**2).mean()))
print('테스트 데이터셋 R2 score', reg.score(X_test,y_test))

테스트 데이터셋 MAE: 0.8961699289210829
테스트 데이터셋 MSE: 1.6167902262116876
테스트 데이터셋 RMSE: 1.2715306627099827
테스트 데이터셋 R2 score 0.3088992786854684


### 회귀의 성능 지표 vs. 분류의 성능 지표
- 분류 - #3
  - 정확도, accuracy
  - 정밀도, precision
  - 재현율, recall
  - F1-score

  - sklearn.metrics 활용하여 얻을 수 있음
    - accuracy_score(y_test, y_pred)
    - precision_score(y_test, y_pred)
    - recall_score(y_test, y_pred)
    - f1_score(y_test, y_pred)
    - classfication_report(y_test, y_pred)

    *(y_test, y_pred) 순서 주의

### 정확도 (accuracy)
- 전체 테스트 샘플 중에서 정확히 분류한 비율
- 절대 틀리면 안되는 경우 *p.16

### 재현율 (recall)
- 실제 1인 샘플 중에서 정확히 분류된(분류기가 1이라고 예측한) 비율

### 정밀도 (precision)
- 1이라고 예측한 샘플 중에서 정확히 분류한(실제 1인) 비율

### F1 score
- 재현율과 정밀도의 조화 평균
- 재현율과 정밀도는 반대의 특성을 갖고 있기 때문에 균형이 중요

### 다중 클래스 분류의 경우?
- 클래스 별 기준의 평균으로 계산

## 3. 분류에서의 성능 지표

### breast_cancer

In [27]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X, y = load_breast_cancer(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=1234)
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

clf = LogisticRegression(random_state=1234, solver='sag', penalty='l2', C=10,
                         max_iter=10000, multi_class='multinomial')
clf.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(C=10, max_iter=10000, multi_class='multinomial',
                   random_state=1234, solver='sag')

In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

y_pred_test = clf.predict(X_test_scaled)

print(f'정확도: {(y_pred_test == y_test).mean()}')
print(f'정확도: {clf.score(X_test_scaled, y_test)}')
print(f'정확도: {accuracy_score(y_test, y_pred_test)}')

정확도: 0.9627659574468085
정확도: 0.9627659574468085
정확도: 0.9627659574468085


In [30]:
print(f'정밀도: {precision_score(y_test, y_pred_test)}')

정밀도: 0.9508196721311475


In [31]:
print(f'재현율: {recall_score(y_test, y_pred_test)}')

재현율: 0.9914529914529915


In [32]:
print(f'F1 score: {f1_score(y_test, y_pred_test)}')

F1 score: 0.9707112970711297


In [35]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.98      0.92      0.95        71
           1       0.95      0.99      0.97       117

    accuracy                           0.96       188
   macro avg       0.97      0.95      0.96       188
weighted avg       0.96      0.96      0.96       188



### iris

In [42]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=1234)

scaler = MinMaxScaler() # 객체 생성
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

clf = LogisticRegression(random_state=1234, solver='sag', penalty='l2', C=10,
                         max_iter=10000, multi_class='multinomial')
clf.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(C=10, max_iter=10000, multi_class='multinomial',
                   random_state=1234, solver='sag')

In [38]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

y_pred_test = clf.predict(X_test_scaled)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        17
           1       0.94      0.94      0.94        17
           2       0.94      0.94      0.94        16

    accuracy                           0.96        50
   macro avg       0.96      0.96      0.96        50
weighted avg       0.96      0.96      0.96        50



### tips

In [39]:
import pandas as pd

tips = pd.read_csv("./tips.csv")

tips.tip=pd.cut(tips['tip'],bins=3,labels=['low','midde','high'])
tips['smoker']=tips['smoker'].map({'No':0,"Yes":1})
tips['day']=tips['day'].map({'Thur':0,"Fri":1,"Sat":2,"Sun":3})
tips['time']=tips['time'].map({'Lunch':0,"Dinner":1})

X_train, X_test, y_train, y_test = train_test_split(tips.drop(columns='tip'), tips['tip'], test_size=0.33, random_state=1234)

In [40]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

clf = LogisticRegression(random_state=1234, solver='sag', penalty='l2', C=10,
                         max_iter=10000, multi_class='multinomial')
clf.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(C=10, max_iter=10000, multi_class='multinomial',
                   random_state=1234, solver='sag')

In [41]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

y_pred_test = clf.predict(X_test_scaled)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

        high       0.50      0.50      0.50         2
         low       0.89      0.89      0.89        66
       midde       0.38      0.38      0.38        13

    accuracy                           0.80        81
   macro avg       0.59      0.59      0.59        81
weighted avg       0.80      0.80      0.80        81

